In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, ReLU
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

# 데이터 디렉토리 설정
train_dir = '/content/drive/My Drive/dataset/Train'
test_dir = '/content/drive/My Drive/dataset/Test'

# .ipynb_checkpoints를 제외한 클래스 디렉토리 목록 생성
train_classes = [folder for folder in os.listdir(train_dir)
                 if os.path.isdir(os.path.join(train_dir, folder)) and folder != '.ipynb_checkpoints']

test_classes = [folder for folder in os.listdir(test_dir)
                if os.path.isdir(os.path.join(test_dir, folder)) and folder != '.ipynb_checkpoints']

print("Filtered Train Classes:", train_classes)
print("Filtered Test Classes:", test_classes)

# 데이터 증강 및 로드
data_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

train_data = data_gen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=8,
    class_mode='categorical',  # 다중 클래스 분류
    classes=train_classes  # 필터링된 클래스 사용
)

test_data = data_gen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=8,
    class_mode='categorical',  # 다중 클래스 분류
    shuffle=False,
    classes=test_classes  # 필터링된 클래스 사용
)

# ResNet50 모델 정의 (처음부터 학습)
base_model = ResNet50(
    include_top=False,
    weights=None,  # 사전 학습된 가중치 사용 안 함
    input_shape=(128, 128, 3)
)

# 모델 구조 커스터마이징
x = base_model.output
x = Flatten()(x)
x = Dense(256)(x)
x = ReLU()(x)
x = Dense(len(train_classes), activation='softmax')(x)  # 클래스 수에 맞게 노드 변경
model = Model(inputs=base_model.input, outputs=x)

# 모델 컴파일
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',  # 다중 클래스 분류 손실 함수
    metrics=['accuracy']
)

# 모델 학습
history = model.fit(
    train_data,
    epochs=5,
    batch_size=8
)

# 모델 평가
test_loss, test_accuracy = model.evaluate(test_data)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


Filtered Train Classes: ['Class1']
Filtered Test Classes: ['Class1']
Found 1 images belonging to 1 classes.
Found 1 images belonging to 1 classes.
Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/losses/losses.py:27: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored

1/1 ━━━━━━━━━━━━━━━━━━━━ 54s 54s/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 17s 17s/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 4/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 5/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 1.0000 - loss: 0.0000e+00
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 1.0000 - loss: 0.0000e+00
Test Loss: 0.0
Test Accuracy: 1.0
